In [1]:
import librosa   #for audio processing
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile#for audio
from scipy.io.wavfile import write
import sounddevice as sd
import soundfile as sf
import cv2

In [2]:
fs = 44100  # Sample rate
seconds = 1  # Duration of recording

print('Start recording')
myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
write('./voice/output.wav', fs, myrecording)  # Save as WAV file
print('Done recording')

Start recording
Done recording


In [3]:
filename = './voice/output.wav'
# Extract data and sampling rate from file
# data, fs = sf.read(filename, dtype='float32') 
# print(fs, data.size)
# sd.play(data, fs)
# status = sd.wait(seconds)  # Wait until file is done playing

In [4]:
sample, sample_rate = librosa.load(filename, sr=16000)

sample = librosa.resample(sample, sample_rate, 8000)
print(sample_rate, sample.size)

16000 8000


In [5]:
def process_voice(filename, rate=1600):
    sample, sample_rate = librosa.load(filename, sr=16000)
    sample = librosa.resample(sample, sample_rate, 8000)
    print(sample_rate, sample.size)
    return sample

In [6]:
# loading model to train
from keras.models import load_model

model = load_model('./models/speech.h5')

Using TensorFlow backend.


In [7]:
#moment of the truth
labels=['unknown', "yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

test = sample.reshape(1,len(sample),1)
prob = model.predict(test)
index = np.argmax(prob[0])
print(index)
labels[index]

8


'off'

In [8]:
prediction = ''
action = ''
score = 0
system_control = True

In [9]:
class Volume(object):
    def __init__(self):
        self.level = .5

    def increase(self, amount):
        self.level += amount
        print(f'New level is: {self.level}')

    def decrease(self, amount):
        self.level -= amount
        print(f'New level is: {self.level}')

vol = Volume()

In [10]:
voice_labels = {labels.index('off'): 0,
                 labels.index('down'): 1,
                 labels.index('up'): 2,
                 labels.index('on'): 3}

voice_names = {0: 'off',
                 1: 'down',
                 2: 'up',
                 3: 'on'}

In [11]:
def predict_voice(sample):
    test = sample.reshape(1,len(sample),1)
    prob = model.predict(test)
    print(f'predict: {prob}')
    index = np.argmax(prob[0])
    score = float("%0.2f" % (max(prob[0]) * 100))
    print(labels[index])
   
    return labels[index], score

predict_voice(sample)

predict: [[0.03868394 0.04181247 0.03434022 0.03003345 0.07996186 0.04896568
  0.03151544 0.11510051 0.4556622  0.09807139 0.02585287]]
off


('off', 45.57)

In [12]:
# Keyboard OP
k = cv2.waitKey(10)
# if k == 27:  # press ESC to exit all windows at any time
#     break
if k == 32:     # If space bar pressed
    print('Ready to play')
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load('yeah.mp3')
    pygame.mixer.music.set_volume(vol.level)
    pygame.mixer.music.play()
    pygame.mixer.music.set_pos(50)
    pygame.mixer.music.pause()

    sample = process_voice(filename, rate=1600)
    prediction, score = predict_voice(sample)

    if system_control == True:
        if prediction == 'on':
            try:
                action = "Music on"

                # sonos.play()
                pygame.mixer.music.unpause()
            # Turn off smart home actions if devices are not responding
            except ConnectionError:
                system_control = False
                pass

        elif prediction == 'off':
            try:
                action = 'Music off'
                # b.set_light(6, off_command)
                # sonos.pause()
                pygame.mixer.music.pause()
            except ConnectionError:
                system_control = False
                pass

        elif prediction == 'down':
            try:
                action = 'Volume down'
                # sonos.volume -= 15
                vol.decrease(0.2)
                pygame.mixer.music.set_volume(vol.level)
            except ConnectionError:
                system_control = False
                pass

        elif prediction == 'Okay':
            try:
                action = 'Volume up'
                # sonos.volume += 15
                vol.increase(0.2)
                pygame.mixer.music.set_volume(vol.level)
            except ConnectionError:
                system_control = False
                pass

        #             elif prediction == 'Peace':
        #                 try:
        #                     action = ''
        #                 except ConnectionError:
        #                     smart_home = False
        #                     pass

        else:
            pass


In [14]:
def record():
    fs = 44100  # Sample rate
    seconds = 1  # Duration of recording

    print('Start recording')
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    write('./voice/output.wav', fs, myrecording)  # Save as WAV file
    print('Done recording')
    return myrecording

In [13]:
i = 0 
while i <=1:
    i += 1
    raw_record = record() #recording and return raw file
    record = process_voice(raw_record, rate=1600)
    sd.play(record, 16000)
    status = sd.wait(1)
    

0
1
2
3
4
5
